In [ ]:
!pip install stanza

In [ ]:
# mount google drive
from google.colab import drive
import sys

drive.mount('/content/drive')
sys.path.append(PATH_TO_DIRECTORY)
%cd PATH_TO_DIRECTORY

Mounted at /content/drive
/content/drive/MyDrive/Research/FairyTaleQA_2


In [ ]:
import pandas as pd
import copy
import csv
import re
import sys
import os
import stanza
from math import isnan


nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')


In [ ]:
def is_nan(x):
    return isinstance(x, float) and isnan(x)

In [ ]:
# uses https://github.com/uclanlp/corefBias/blob/master/WinoBias/wino/word_swapper.py
# assumes generalized_swaps.txt and extra_gendered_words.txt are in the subdirectory ./pertubation_dicts

def rule_based_translation(input, output):
  swaps = open("./perturbation_dicts/generalized_swaps.txt").readlines()
  swap_dict = {}
  for line in swaps:
    tabs = line.split("\t")
    swap_dict[tabs[-2].strip()] = tabs[-1].strip()

  swaps = open("./perturbation_dicts/extra_gendered_words.txt").readlines()
  for line in swaps:
    tabs = line.split("\t")
    swap_dict[tabs[-2].strip()] = tabs[-1].strip()
    swap_dict[tabs[-1].strip()] = tabs[-2].strip()


  fairytale_data = pd.read_csv(input)

  for index, row in fairytale_data.iterrows():

    for attr in ['question', 'answer1', 'answer4', 'text', 'answer5', 'answer2', 'answer6', 'answer3']:

      if row[attr] is not None and not is_nan(row[attr]):
        text = nlp(row[attr])
        perturbed_text = ''

        for sentence in text.sentences:

          for word_data in sentence.words:
            word = word_data.text.lower()
            swap = None

            if word in swap_dict:
              swap = swap_dict[word]

            # check for pronoun or possesive use of 'her'
            if word == 'her':
              if word_data.xpos == 'PRP':
                swap = 'him'
              else:
                swap = 'his'

            # check for possesive adjective use of 'his'
            if word == 'his':
              if word_data.xpos == 'PRP$':
                swap = 'her'
              else:
                swap = 'hers'

            # handles capitalization and appending the word to the text
            if swap is not None and word_data.text.isupper():
              swap = swap[0].upper() + swap[1:]
            if swap is None:
              perturbed_text += word + ' '
            else:
              perturbed_text += swap + ' '

        fairytale_data[attr][index]= perturbed_text

  fairytale_data.to_csv(output, index=False)



In [ ]:
rule_based_translation('./fairytaleqa_preprocessed/fairytaleqa_preprocessed_train.csv', './rule_based_translation/fairytaleqa_preprocessed_train.csv')

<ipython-input-5-0f636be74c08>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fairytale_data[attr][index]= perturbed_text
